# Functions precipitation
***

In this notebook we save functions we have used in the precipitation exercises, so that you can import them directly in any other notebook and use them. 

To load these functions in other notebook, you must use a so-called magic function in _Python_ terminology:
```Python
%run funciones_precipitacion.ipynb
```

The previous code only works if `functions_precipitation.ipynb` is in the same folder as the notebook in which you want to load it. If that is not the case, you must use package `os` to define the folder in which `functions_precipitation.ipynb` is located.
```Python
import os
os.chdir('path where `functions_precipitation.ipynb' is)
%run functions_precipitation.ipynb
```

__Table of contentes__<br>

__[IDW interpolation](#IDW-interpolation)__<br>

__[Linear regression](#Linear-regression)__<br>

__[IDF curves](#IDF-curves)__<br>

__[Read & write](#Read-&-write)__<br>

In [3]:
import numpy as np

import pandas as pd

#from matplotlib import pyplot as plt
#import seaborn as sns
#sns.set()

from scipy.stats import genextreme
from scipy.optimize import curve_fit

### IDW interpolation

In [1]:
def IDW(x, y, stnX, stnY, stnP, b=-2):
    """Interpolate by the inverse distance weighted method
    
    Parameters:
    ----------
    x:       float. Coordenate X of the target point
    y:       float. Coordenate Y of the target point
    stnX:    Series. Coordenates X of the gages
    stnY:    Series. Coordenates Y of the gages
    stnP:    Series. Observed precipitation in the gages
    b:       int. Exponent in the inverse distance
    
    Returns:
    --------
    p:       float. Precipitation interpolated for point (x, y)
    """
    
    # distance to the target point
    distX = x - stnX                    # distancia en el eje X
    distY = y - stnY                    # distancia en el eje X
    dist = np.sqrt(distX**2 + distY**2) # distancia total
    # inverse of the distance
    idw = dist**b
    # interpolate
    p = np.sum(idw / np.sum(idw) * stnP)
    
    return round(p, 1)

### Linear regression

In [79]:
def linear_reg(x, b):
    """Linear regression with no intecept
    
    y = b * x   
    
    Input:
    ------
    x:         float. Independet value
    b:         float. Slope of the linear regression
    
    Output:
    -------
    y:         float. Regressed value"""
    
    y = b * x
    return y

### IDF curves

In [31]:
def IDF_type_I(x, b, c, d, e):
    """Estimate precipitation intensity fiven a return period and a storm duration using the analytical IDF curve type I:
    
    I = d * R + e / (D + c)**b.    
    
    Input:
    ------
    x:         list [2x1]. Values of return period (years) and duration (h)
    b:         float. Parameter of the IDF curve
    c:         float. Parameter of the IDF curve
    d:         float. Parameter of the IDF curve
    e:         float. Parameter of the IDF curve
    
    Output:
    -------
    I:         float. Precipitation intensity (mm/h)"""
    
    a = d * x[0] + e
    I = a / (x[1] + c)**b
    return I

In [31]:
def IDF_type_II(x, b, c, d, e):
    """Estimate precipitation intensity fiven a return period and a storm duration using the analytical IDF curve type II:
    
    I = d * R + e / (D**b + c).    
    
    Input:
    ------
    x:         list [2x1]. Values of return period (years) and duration (h)
    b:         float. Parameter of the IDF curve
    c:         float. Parameter of the IDF curve
    d:         float. Parameter of the IDF curve
    e:         float. Parameter of the IDF curve
    
    Output:
    -------
    I:         float. Precipitation intensity (mm/h)"""
    
    a = d * x[0] + e
    I = a / (x[1]**b + c)
    return I

In [31]:
def IDF_type_III(x, b, c, d, e):
    """Estimate precipitation intensity fiven a return period and a storm duration using the analytical IDF curve type III:
    
    I = d * R**e / (D + c)**b.    
    
    Input:
    ------
    x:         list [2x1]. Values of return period (years) and duration (h)
    b:         float. Parameter of the IDF curve
    c:         float. Parameter of the IDF curve
    d:         float. Parameter of the IDF curve
    e:         float. Parameter of the IDF curve
    
    Output:
    -------
    I:         float. Precipitation intensity (mm/h)"""
    
    a = d * x[0]**e 
    I = a / (x[1] + c)**b
    return I

In [31]:
def IDF_type_IV(x, b, c, d, e):
    """Estimate precipitation intensity fiven a return period and a storm duration using the analytical IDF curve type IV:
    
    I = d * R**e / (D**b + c).    
    
    Input:
    ------
    x:         list [2x1]. Values of return period (years) and duration (h)
    b:         float. Parameter of the IDF curve
    c:         float. Parameter of the IDF curve
    d:         float. Parameter of the IDF curve
    e:         float. Parameter of the IDF curve
    
    Output:
    -------
    I:         float. Precipitation intensity (mm/h)"""
    
    a = d * x[0]**e
    I = a / (x[1]**b + c)
    return I 

### Read & write

In [2]:
def read_ascii(filename, datatype='float'):
    """Import an ASCII file. Data is saved as a 2D numpy array and the attributes as integers or floating point numbers.
    
    Parameters:
    -----------
    filename:     string. Name (including path and extension) of the ASCII file
    datatype:     string. Type of data inside the file
    
    Output:
    -------
    data:         array[nrows,ncols]. The data in the map
	attributes:   list. A list of six attributes: 
						ncols:        int. Number of columns
						nrows:        int. Number of rows
						xllcorner:    float. X coordinate of the left lower corner
						yllcorner:    float. Y coordinate of the left lower corner
						cellsize:     int. Spatial discretization
						NODATA_value: float. Value representing no data
    """
    
    with open(filename, 'r+') as file:
        # import all the lines in the file
        asc = file.readlines()
        # extract attributes
        ncols = int(asc[0].split()[1])
        nrows = int(asc[1].split()[1])
        xllcorner = float(asc[2].split()[1])
        yllcorner = float(asc[3].split()[1])
        cellsize = int(asc[4].split()[1])
        NODATA_value = float(asc[5].split()[1])
        attributes = [ncols, nrows, xllcorner, yllcorner, cellsize, NODATA_value]
        # extract data
        data = np.zeros((nrows, ncols))
        for i in range(nrows):
            data[i, :] = asc[i + 6].split()
        data[data == NODATA_value] = np.nan
        #data = np.ma.masked_invalid(data)
        data = data.astype(datatype)
    file.close()    
    
    return data, attributes